In [1]:
import glob
glob.glob('./data/*.csv.zip')

['./data/train.csv.zip', './data/test.csv.zip']

In [2]:
import os
import pandas as pd

In [3]:
os.system('unzip ./data/train.csv.zip')
df_train = pd.read_csv('train.csv')
df_train.head()

,id,tid1,tid2,title1_zh,title2_zh,title1_en,title2_en,label
0,0,0,1,2017养老保险又新增两项，农村老人人人可申领，你领到了吗,警方辟谣“鸟巢大会每人领5万” 仍有老人坚持进京,There are two new old-age insurance benefits f...,"Police disprove ""bird's nest congress each per...",unrelated
1,3,2,3,"""你不来深圳，早晚你儿子也要来""，不出10年深圳人均GDP将超香港",深圳GDP首超香港？深圳统计局辟谣：只是差距在缩小,"""If you do not come to Shenzhen, sooner or lat...",Shenzhen's GDP outstrips Hong Kong? Shenzhen S...,unrelated
2,1,2,4,"""你不来深圳，早晚你儿子也要来""，不出10年深圳人均GDP将超香港",GDP首超香港？深圳澄清：还差一点点……,"""If you do not come to Shenzhen, sooner or lat...",The GDP overtopped Hong Kong? Shenzhen clarifi...,unrelated
3,2,2,5,"""你不来深圳，早晚你儿子也要来""，不出10年深圳人均GDP将超香港",去年深圳GDP首超香港？深圳统计局辟谣：还差611亿,"""If you do not come to Shenzhen, sooner or lat...",Shenzhen's GDP topped Hong Kong last year? She...,unrelated
4,9,6,7,"""用大蒜鉴别地沟油的方法,怎么鉴别地沟油",吃了30年食用油才知道，一片大蒜轻松鉴别地沟油,"""How to discriminate oil from gutter oil by me...",It took 30 years of cooking oil to know that o...,agreed


In [4]:
empty_title = ((df_train['title1_zh'].isnull()) \
         | (df_train['title2_zh'].isnull()) \
         | (df_train['title2_zh'] == '') \
         | (df_train['title2_zh'] == '0'))
empty_title

0         False
1         False
2         False
3         False
4         False
          ...  
320547    False
320548    False
320549    False
320550    False
320551    False
Length: 320552, dtype: bool

In [5]:
df_train = df_train[~empty_title]

MAX_LENGTH = 30
df_train = df_train[~(df_train.title1_zh.apply(lambda x: len(x)) > MAX_LENGTH)]
df_train = df_train[~(df_train.title2_zh.apply(lambda x: len(x)) > MAX_LENGTH)]

SAMPLE_RATE = 0.01
df_train = df_train.sample(frac=SAMPLE_RATE, random_state=2020)
df_train = df_train.reset_index()
df_train = df_train.loc[:, ['title1_zh', 'title2_zh', 'label']]
df_train.columns = ['text_a', 'text_b', 'label']
df_train.head()

,text_a,text_b,label
0,8张图片能证明张柏芝不会和谢霆锋复婚,看看张柏芝散发出的母爱 就知道所有新恋情都是谣言,unrelated
1,普洱茶致癌、中式咸鱼致癌？院士专家解读2017年食品安全热点,邢台金沙河挂面掺胶？谣言！这些有关食品的说法都是谣言不要信！,unrelated
2,联想的5G投票最后为何没给华为，是什么意思？,联想投票反对预装国产系统 联想辟谣：我们反对的只是双系统方案,unrelated
3,娱乐猛料：唐嫣罗晋被曝分手，网友说又被玩了！老铁们你们怎么看,玩雪伤眼？兰州拉面致癌？一月“十大谣言”出炉，你上当了吗？,unrelated
4,抗流感病毒药奥司他韦多地断货！国产药正在加速上市,流感严重，奥司他韦卖断货,agreed


In [6]:
df_train.to_csv('train.tsv', sep='\t', index=False)
print('train samples: ', len(df_train))

train samples:  2657


In [7]:
df_train.label.value_counts() / len(df_train)

unrelated    0.674068
agreed       0.297704
disagreed    0.028227
Name: label, dtype: float64

In [8]:
os.system('unzip ./data/test.csv.zip')
df_test = pd.read_csv('test.csv')
df_test = df_test.loc[:, ['title1_zh', 'title2_zh', 'id']]
df_test.columns = ['text_a', 'text_b', 'Id']
df_test.to_csv('test.tsv', sep='\t', index=False)

print('test samples: ', len(df_test))
df_test.head()



test samples:  80126


,text_a,text_b,Id
0,萨拉赫人气爆棚!埃及总统大选未参选获百万选票 现任总统压力山大,辟谣！里昂官方否认费基尔加盟利物浦，难道是价格没谈拢？,321187
1,萨达姆被捕后告诫美国的一句话，发人深思,10大最让美国人相信的荒诞谣言，如蜥蜴人掌控着美国,321190
2,萨达姆此项计划没有此国破坏的话，美国还会对伊拉克发动战争吗,萨达姆被捕后告诫美国的一句话，发人深思,321189
3,萨达姆被捕后告诫美国的一句话，发人深思,被绞刑处死的萨达姆是替身？他的此男人举动击破替身谣言！,321193
4,萨达姆被捕后告诫美国的一句话，发人深思,中国川贝枇杷膏在美国受到热捧？纯属谣言！,321191


In [9]:
ratio = len(df_test) / len(df_train)
print("測試集樣本數 / 訓練集樣本數 = {:.1f} 倍".format(ratio))

測試集樣本數 / 訓練集樣本數 = 30.2 倍


In [10]:
"""
實作一個可以用來讀取訓練 / 測試集的 Dataset，這是你需要徹底了解的部分。
此 Dataset 每次將 tsv 裡的一筆成對句子轉換成 BERT 相容的格式，並回傳 3 個 tensors：
- tokens_tensor：兩個句子合併後的索引序列，包含 [CLS] 與 [SEP]
- segments_tensor：可以用來識別兩個句子界限的 binary tensor
- label_tensor：將分類標籤轉換成類別索引的 tensor, 如果是測試集則回傳 None
"""
import torch
import pysnooper
from torch.utils.data import Dataset


class FakeNewsDataset(Dataset):
    def __init__(self, mode, tokenizer):
        assert mode in ['train', 'dev', 'test']
        self.mode = mode

        self.df = pd.read_csv('%s.tsv' % mode, sep='\t').fillna('')
        self.len = len(self.df)
        self.label_map = {'agreed': 0, 'disagreed': 1, 'unrelated': 2}
        self.tokenizer = tokenizer

    # @pysnooper.snoop()
    def __getitem__(self, idx):
        if self.mode == 'test':
            text_a, text_b = self.df.iloc[idx, :2].values
            label_tensor = None
        else:
            text_a, text_b, label = self.df.iloc[idx, :].values
            label_id = self.label_map[label]
            label_tensor = torch.tensor(label_id)
        
        word_pieces = ['[CLS]']
        tokens_a = self.tokenizer.tokenize(text_a)
        word_pieces += tokens_a + ['[SEP]']
        len_a = len(tokens_a)

        tokens_b = self.tokenizer.tokenize(text_b)
        word_pieces += tokens_b + ['[SEP]']
        len_b = len(word_pieces) - len_a

        ids = self.tokenizer.convert_tokens_to_ids(word_pieces)
        tokens_tensor = torch.tensor(ids)

        segments_tensor = torch.tensor([0] * len_a + [1] * len_b, dtype=torch.long)

        return (tokens_tensor, segments_tensor, label_tensor)

    def __len__(self):
        return self.len


In [11]:

import torch
from transformers import BertTokenizer
from IPython.display import clear_output

PRETRAINED_MODEL_NAME = "bert-base-chinese"  # 指定繁簡中文 BERT-BASE 預訓練模型

# 取得此預訓練模型所使用的 tokenizer
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)
trainset = FakeNewsDataset('train', tokenizer)

In [12]:
sample_idx = 0
text_a, text_b, label = trainset.df.iloc[sample_idx, :].values
tokens_tensor, segments_tensor, label_tensor = trainset[sample_idx]

tokens = tokenizer.convert_ids_to_tokens(tokens_tensor.tolist())
combined_text = ''.join(tokens)

print(f"""[原始文本]
句子 1：{text_a}
句子 2：{text_b}
分類  ：{label}

--------------------

[Dataset 回傳的 tensors]
tokens_tensor  ：{tokens_tensor}

segments_tensor：{segments_tensor}

label_tensor   ：{label_tensor}

--------------------

[還原 tokens_tensors]
{combined_text}
""")


[原始文本]
句子 1：8张图片能证明张柏芝不会和谢霆锋复婚
句子 2：看看张柏芝散发出的母爱 就知道所有新恋情都是谣言
分類  ：unrelated

--------------------

[Dataset 回傳的 tensors]
tokens_tensor  ：tensor([ 101,  129, 2476, 1745, 4275, 5543, 6395, 3209, 2476, 3377, 5698,  679,
         833, 1469, 6468, 7447, 7226, 1908, 2042,  102, 4692, 4692, 2476, 3377,
        5698, 3141, 1355, 1139, 4638, 3678, 4263, 2218, 4761, 6887, 2792, 3300,
        3173, 2605, 2658, 6963, 3221, 6469, 6241,  102])

segments_tensor：tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

label_tensor   ：2

--------------------

[還原 tokens_tensors]
[CLS]8张图片能证明张柏芝不会和谢霆锋复婚[SEP]看看张柏芝散发出的母爱就知道所有新恋情都是谣言[SEP]



In [13]:
"""
實作可以一次回傳一個 mini-batch 的 DataLoader
這個 DataLoader 吃我們上面定義的 `FakeNewsDataset`，
回傳訓練 BERT 時會需要的 4 個 tensors：
- tokens_tensors  : (batch_size, max_seq_len_in_batch)
- segments_tensors: (batch_size, max_seq_len_in_batch)
- masks_tensors   : (batch_size, max_seq_len_in_batch)
- label_ids       : (batch_size)
"""
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence


# 這個函式的輸入 `samples` 是一個 list，裡頭的每個 element 都是
# 剛剛定義的 `FakeNewsDataset` 回傳的一個樣本，每個樣本都包含 3 tensors：
# - tokens_tensor
# - segments_tensor
# - label_tensor
# 它會對前兩個 tensors 作 zero padding，並產生前面說明過的 masks_tensors
def create_mini_batch(samples):
    tokens_tensors = [s[0] for s in samples]
    segments_tensors = [s[1] for s in samples]

    if samples[0][2] is not None:
        label_ids = torch.stack([s[2] for s in samples]) # Concatenates sequence of tensors along a new dimension.
    else:
        label_ids = None

    # Pad a list of variable length Tensors with padding_value
    # This function returns a Tensor of size T x B x * or B x T x * where T is the length of the longest sequence. 
    # This function assumes trailing dimensions and type of all the Tensors in sequences are same.
    tokens_tensors = pad_sequence(tokens_tensors, batch_first=True) 
    segments_tensors = pad_sequence(segments_tensors, batch_first=True)

    # attention masks，將 tokens_tensors 裡頭不為 zero padding
    # 的位置設為 1 讓 BERT 只關注這些位置的 tokens
    masks_tensors = torch.zeros(tokens_tensors.shape, dtype=torch.long)
    masks_tensors = masks_tensors.masked_fill(tokens_tensors != 0, 1)

    return tokens_tensors, segments_tensors, masks_tensors, label_ids


BATCH_SIZE = 64
trainloader = DataLoader(trainset, batch_size=BATCH_SIZE, collate_fn=create_mini_batch)


In [14]:
data = next(iter(trainloader))
tokens_tensors, segments_tensors, masks_tensors, label_ids = data
print(f"""
tokens_tensors.shape   = {tokens_tensors.shape} 
{tokens_tensors}
------------------------
segments_tensors.shape = {segments_tensors.shape}
{segments_tensors}
------------------------
masks_tensors.shape    = {masks_tensors.shape}
{masks_tensors}
------------------------
label_ids.shape        = {label_ids.shape}
{label_ids}
""")


tokens_tensors.shape   = torch.Size([64, 62]) 
tensor([[ 101,  129, 2476,  ...,    0,    0,    0],
        [ 101, 3249, 3827,  ...,  102,    0,    0],
        [ 101, 5468, 2682,  ...,    0,    0,    0],
        ...,
        [ 101, 3557, 3207,  ...,    0,    0,    0],
        [ 101, 8267, 2259,  ...,    0,    0,    0],
        [ 101, 1600, 5763,  ...,    0,    0,    0]])
------------------------
segments_tensors.shape = torch.Size([64, 62])
tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 1, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])
------------------------
masks_tensors.shape    = torch.Size([64, 62])
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])
------------------------
label_ids.shape        

In [15]:
from transformers import BertForSequenceClassification

PRETRAINED_MODEL_PATH = './bert-base-chinese/'
NUM_LABELS = 3

model = BertForSequenceClassification.from_pretrained(PRETRAINED_MODEL_PATH, num_labels = NUM_LABELS)
clear_output()

print('{0:10}{1:15}'.format('name', 'module'))
for name, module in model.named_children():
    if name == 'bert':
        for n, _ in module.named_children():
            print(f'{name}:{n}')
    else:
        print('{:15}{}'.format(name, module))

name      module         
bert:embeddings
bert:encoder
bert:pooler
dropout        Dropout(p=0.1, inplace=False)
classifier     Linear(in_features=768, out_features=3, bias=True)


In [16]:
"""
定義一個可以針對特定 DataLoader 取得模型預測結果以及分類準確度的函式
在將 `tokens`、`segments_tensors` 等 tensors
丟入模型時，強力建議指定每個 tensor 對應的參數名稱，以避免 HuggingFace
更新 repo 程式碼並改變參數順序時影響到我們的結果。
"""
def get_predictions(model, dataloader, compute_acc=False):
    predictions = None
    correct = 0
    total = 0

    with torch.no_grad():
        for data in dataloader:
            if next(model.parameters()).is_cuda: # check if the model is on cuda
                data = [t.to(torch.device('cuda:0')) for t in data if t is not None]

            tokens_tensors, segments_tensors, masks_tensors = data[:3]
            outputs = model(input_ids=tokens_tensors, 
                            token_type_ids=segments_tensors, 
                            attention_mask=masks_tensors)
            logits = outputs[0]
            _, pred = torch.max(logits.data, 1)

            if compute_acc:
                labels = data[3]
                total += labels.size(0)
                correct += (pred == labels).sum().item()

            if predictions is None:
                predictions = pred
            else:
                predictions = torch.cat((predictions, pred))

        if compute_acc:
            acc = correct / total
            return predictions, acc

        return predictions


In [17]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('device: ', device)
model = model.to(device)
_, acc = get_predictions(model, trainloader, compute_acc=True)
print('classification acc: ', acc)


device:  cuda:0
classification acc:  0.48550997365449755


In [18]:
def get_learnable_params(module):
    return [p for p in module.parameters() if p.requires_grad]

model_params = get_learnable_params(model)
clf_params = get_learnable_params(model.classifier)


print(f"""
整個分類模型的參數量：{sum(p.numel() for p in model_params)}
線性分類器的參數量：{sum(p.numel() for p in clf_params)}
""")



整個分類模型的參數量：102269955
線性分類器的參數量：2307



In [19]:
%%time
import torch.optim as optim

model.train()
optimizer = optim.Adam(model.parameters(), lr=1e-5)

EPOCHS = 6
for epoch in range(EPOCHS):
    running_loss = 0.0
    for data in trainloader:
        tokens_tensors, segments_tensors, masks_tensors, labels = [t.to(device) for t in data]

        optimizer.zero_grad()
        outputs = model(input_ids=tokens_tensors, 
                        token_type_ids=segments_tensors, 
                        attention_mask=masks_tensors,
                        labels=labels)
        loss = outputs[0]
        loss.backward()
        optimizer.step()

        running_loss += loss

    _, acc = get_predictions(model, trainloader, compute_acc=True)

    print('[epoch %d] loss: %.3f, acc: %.3f' %(epoch + 1, running_loss, acc))

[epoch 1] loss: 24.194, acc: 0.826
[epoch 2] loss: 18.037, acc: 0.881
[epoch 3] loss: 14.730, acc: 0.912
[epoch 4] loss: 10.960, acc: 0.937
[epoch 5] loss: 8.292, acc: 0.951
[epoch 6] loss: 7.400, acc: 0.967
CPU times: user 1min 13s, sys: 23.7 s, total: 1min 37s
Wall time: 1min 37s


In [20]:
testset = FakeNewsDataset('test', tokenizer=tokenizer)
testloader = DataLoader(testset, batch_size=256, collate_fn=create_mini_batch)

predictions = get_predictions(model, testloader)
index_map = {v: k for k, v in testset.label_map.items()}

df = pd.DataFrame({"Category" : predictions.tolist()})
df['Category'] = df.Category.apply(lambda x: index_map[x])
df_pred = pd.concat([testset.df.loc[:, 'Id'], df.loc[:, 'Category']], axis = 1)
df_pred.to_csv('bert_1_prec_training_samples.csv', index=False)
df_pred.head()

,Id,Category
0,321187,unrelated
1,321190,unrelated
2,321189,unrelated
3,321193,unrelated
4,321191,unrelated


In [22]:
predictions = get_predictions(model, trainloader)
df = pd.DataFrame({'predicted': predictions.tolist()})
df['predicted'] = df.predicted.apply(lambda x: index_map[x])
df1 = pd.concat([trainset.df, df.loc[:, 'predicted']], axis=1)
disagreed_tp = ((df1.label == 'disagreed') & \
     (df1.label == df1.predicted) & \
     (df1.text_a.apply(lambda x: True if len(x) < 10 else False)))
df1[disagreed_tp].head()

,text_a,text_b,label,predicted
1174,张卫健吸毒被带走,网传张卫健吸毒被捕？发微博力破谣言！,disagreed,disagreed
1365,腾讯即将开始封群,辟谣｜腾讯即将开始封群 最近千万别在群里发任何东西,disagreed,disagreed
1514,第六套人民币亮相了,「辟谣」第六套新版人民币亮相了？别再被假象蒙蔽！,disagreed,disagreed
2369,张卫健吸毒被带走,网爆张卫健吸毒被抓是谣言，爆料者已道歉，并已移交警方做笔录,disagreed,disagreed


In [ ]:
# 安裝 BertViz
import sys
!test -d bertviz_repo || git clone https://github.com/jessevig/bertviz bertviz_repo
if not 'bertviz_repo' in sys.path:
  sys.path += ['bertviz_repo']

# import packages
from transformers import BertTokenizer, BertModel
from bertviz import head_view

# 在 jupyter notebook 裡頭顯示 visualzation 的 helper
def call_html():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              "d3": "https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.8/d3.min",
              jquery: '//ajax.googleapis.com/ajax/libs/jquery/2.0.0/jquery.min',
            },
          });
        </script>
        '''))

clear_output()

In [ ]:
from transformers import BertModel
model_version = 'bert-base-chinese'
finetuned_model = BertModel.from_pretrained(model_version, output_attention=True, state_dict=model.state_dict())

# 兩個句子
sentence_a = "烟王褚时健去世"
sentence_b = "辟谣：一代烟王褚时健安好！"

# 得到 tokens 後丟入 BERT 取得 attention
inputs = tokenizer.encode_plus(sentence_a, sentence_b, return_tensors='pt', add_special_tokens=True)
token_type_ids = inputs['token_type_ids']
input_ids = inputs['input_ids']
attention = finetuned_model(input_ids, token_type_ids=token_type_ids)[-1]
input_id_list = input_ids[0].tolist()
tokens = tokenizer.convert_ids_to_tokens(input_id_list)
call_html()
head_view(attention, tokens)


